In [1]:
from pathlib import Path
import time
import csv
from wwo_hist import retrieve_hist_data
from typing import List, Iterable
import os

In [2]:
API_KEY   = "5ccf4c0bfb8849ddaf8204104250207"

# se __file__ existir, usa a pasta do arquivo; senão, usa a pasta atual
BASE_ROOT = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()

BASE_DIR  = BASE_ROOT / ".." / ".." / "Bases" / "Clima" / "worldWeatherApi"
DONE_DIR  = BASE_DIR / "done"
DONE_FILE = DONE_DIR / "done.txt"

In [ ]:
from pathlib import Path
from typing  import List, Iterable
from wwo_hist import retrieve_hist_data
import time, os



# ------------------------------------------------------------------ utilidades
def save_lines(lines: Iterable[str], file: Path) -> None:
    file.write_text("\n".join(lines), encoding="utf-8")

def read_lines(file: Path) -> List[str]:
    return file.read_text(encoding="utf-8").splitlines() if file.exists() else []

# ------------------------------------------------------------------ download
def fetch_history(city: str, start: str, end: str, freq: int = 24, pause: float = 2):
    """Baixa os dados da cidade e grava o CSV DENTRO de BASE_DIR."""
    time.sleep(pause)

    cwd = Path.cwd()            # guarda onde eu estava
    try:
        os.chdir(BASE_DIR)      # muda só durante o download
        retrieve_hist_data(
            API_KEY, [city], start, end, freq,
            location_label=False, export_csv=True, store_df=True
        )
    finally:
        os.chdir(cwd)           # volta ao normal

# ------------------------------------------------------------------ orquestra
def process_weather(
    locations: List[str],
    start_date="2025-06-25",
    end_date="2025-07-01",
    freq=24,
    retries=3,
):
    # garante pastas
    DONE_DIR.mkdir(parents=True, exist_ok=True)
    done = set(read_lines(DONE_FILE))

    for city in locations:
        if city in done:
            continue

        for attempt in range(1, retries + 1):
            try:
                fetch_history(city, start_date, end_date, freq)
                done.add(city)
                save_lines(sorted(done), DONE_FILE)
                break
            except Exception as err:
                print(f"Falhou {city} ({attempt}/{retries}): {err}")
                if attempt == retries:
                    print("Desisti dessa cidade.")


In [50]:
cidades = ["Brasil,Marilia"]
process_weather(cidades)



Retrieving weather data for Brasil,Marilia


Currently retrieving data for Brasil,Marilia: from 2025-06-25 to 2025-06-30


C:\Users\johns\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wwo_hist\__init__.py:69: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
C:\Users\johns\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wwo_hist\__init__.py:74: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:00.328313
Currently retrieving data for Brasil,Marilia: from 2025-07-01 to 2025-07-01
Time elapsed (hh:mm:ss.ms) 0:00:00.777023


export Brasil,Marilia completed!


